# Production-Ready Agent Workflow with Flotorch and CrewAI

This notebook demonstrates a comprehensive, production-style workflow. It integrates all the advanced features we've explored into a single, robust application:
- **Remote Agent Configuration**: Uses `FlotorchCrewAIAgent` to load agent personas from the Flotorch UI.
- **Custom Local Tools**: Defines and attaches a custom tool to the agent at runtime.
- **Dual Memory System**: Implements both short-term session memory and long-term external memory.


### Prerequesit
Configure agent, memory provider and API key in Flotroch console (https://console.flotorch.cloud/)

### Viewing logs
Logs can be viewed in logs tab in Flotroch console (https://console.flotorch.cloud/)

## 1. Setup and Imports

The following cells install dependencies, set API credentials, and import all necessary components for handling tools, memory, and remote agent management.

In [ ]:
# install flotorch adk package
%pip install flotorch[crewai]==2.4.0b1

In [ ]:
FLOTORCH_API_KEY = "<flotorch api key>"
FLOTORCH_BASE_URL = "<flotroch gateway base url>" # eg: https://gateway.flotorch.cloud"
AGENT_NAME = "<your agent name>"
MEMORY_PROVIDER = "<your mem0 provider>"
APP_NAME = "app_name"
USER_ID = "user_id"

## 2. Import Required Components

This section imports all the necessary components for building a production-ready medical assistant with Flotorch and CrewAI:

### Flotorch Integrations:
- **`FlotorchCrewAIAgent`**: Loads agent configurations (persona, goals, instructions) from the Flotorch console
- **`FlotorchCrewAISession`**: Manages short-term conversational memory using Flotorch's session storage
- **`FlotorchMemoryStorage`**: Provides long-term memory persistence using Flotorch's memory providers (e.g., Mem0)

### CrewAI Memory Components:
- **`ShortTermMemory`**: Maintains context within the current conversation session
- **`ExternalMemory`**: Stores and retrieves information across sessions for long-term knowledge

### CrewAI Core:
- **`Crew`**: Orchestrates agents, tasks, and memory systems into a cohesive workflow

These components work together to create an intelligent medical assistant that can remember patient information across sessions while maintaining conversational context.

In [ ]:
# Import necessary libraries
from flotorch.crewai.agent import FlotorchCrewAIAgent
from flotorch.crewai.sessions import FlotorchCrewAISession
from flotorch.crewai.memory import FlotorchMemoryStorage
from crewai.memory.short_term.short_term_memory import ShortTermMemory
from crewai.memory.external.external_memory import ExternalMemory
from crewai import Crew

print("Imported necessary libraries successfully")

## 3. Memory Configuration

We configure a **dual-memory system** to give our medical assistant both short-term conversational awareness and long-term patient information persistence.

### Short-Term Memory (Conversational Context)
```python
short_term_storage = FlotorchCrewAISession(...)
short_term_memory = ShortTermMemory(storage = short_term_storage)
```

### External Memory (Long-Term Persistence)
```python
external_storage = FlotorchMemoryStorage(...)
external_memory = ExternalMemory(storage = external_storage)
```
### Configuration Parameters:
- **`name`**: Memory provider name (e.g., "memo-provider") configured in Flotorch
- **`user_id`**: Unique identifier for the patient (e.g., "patient_12345")
- **`app_id`**: Application identifier to separate different use cases

This dual-memory architecture ensures our medical assistant is both contextually aware during conversations and can maintain long-term patient relationships.

In [ ]:
short_term_storage = FlotorchCrewAISession(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL
)

short_term_memory = ShortTermMemory(storage = short_term_storage)

external_storage = FlotorchMemoryStorage(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    name = MEMORY_PROVIDER,
    user_id = USER_ID,
    app_id  = APP_NAME
)

external_memory = ExternalMemory(storage = external_storage)

print("Initialized short-term and external Memories")

## 4. Agent Initialization from Flotorch Console

This section loads the pre-configured medical assistant agent from the Flotorch console, enabling centralized configuration management.

### Creating the Agent Client
```python
flotorch_client = FlotorchCrewAIAgent(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    agent_name=AGENT_NAME,
)
```

### Fetching Agent & Task Configuration
```python
agent = flotorch_client.get_agent()
task = flotorch_client.get_task()
```

**`get_agent()`** retrieves:
- **Role & Goal**: The agent's medical assistant persona and objectives
- **Backstory**: Context about the agent's expertise and approach
- **System Prompt**: Detailed instructions for medical consultations
- **LLM Configuration**: Model selection and parameters (e.g., gpt-4o-mini)

**`get_task()`** retrieves:
- **Description**: What the agent should accomplish (e.g., "Provide medical assistance")
- **Expected Output**: Format and structure of responses
- **Task-Specific Instructions**: Step-by-step consultation workflow

```

This approach separates agent logic (code) from agent behavior (configuration), making it easy to iterate and improve your medical assistant without redeployment.

In [ ]:
flotorch_client = FlotorchCrewAIAgent(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    agent_name=AGENT_NAME,
)

agent = flotorch_client.get_agent()
task = flotorch_client.get_task()

print("Fetched agent and task from Flotorch console")

## 5. Assembling the Complete Crew

This is where we bring everything together! The `Crew` is CrewAI's orchestration layer that coordinates agents, tasks, and memory systems into a cohesive AI application.

### Creating the Crew
```python
crew = Crew(
    agents = [agent],
    tasks = [task],
    short_term_memory = short_term_memory,
    external_memory = external_memory,
    verbose = False
)
```

This crew is now a **production-ready medical assistant** with conversational awareness, persistent memory, and centralized configuration!

In [ ]:
crew = Crew(
    agents = [agent],
    tasks = [task],
    short_term_memory = short_term_memory,
    external_memory= external_memory,
    verbose = False
)

print("Created Crew with agent, task and memories")

## 6. Interactive Chat Interface

### Purpose
Start an interactive conversation with the medical assistant.

### Features
- **Continuous Conversation**: Chat until you type "exit"
- **Automatic Memory**: Patient data saved automatically
- **Session Persistence**: Conversation context maintained throughout

### Usage Instructions
1. Run the cell below
2. Enter your messages when prompted
3. Type `exit` to end the conversation

### Example Interaction Flow
```
You: Hello
Dr. Hello! How can I assist you with your health today? Before we get started, may I have your name, age, and gender?

You: My name is John, I'm 35, male
Dr. AI: Thank you, John. What brings you here today?

You: I have a headache
Dr. AI: I understand. Can you describe the headache? When did it start?...
```

### Behind the Scenes
- Agent collects: Name, Age, Gender
- Then gathers: Symptoms, Medical History, Medications, Allergies
- Stores everything in structured memory format
- Next visit: Automatically recognizes "John" and retrieves history

In [ ]:
while True:

    user_query = input("user: ")

    if user_query.lower().strip() == "exit":
        break
    response = crew.kickoff(inputs = {"query":user_query})
    print(f"Assistant:{response.raw}")

---
## Summary

### What We Built

A production-ready medical assistant that:

### Recognizes Returning Patients
- Automatically identifies patients by name
- Retrieves complete medical history
- Shows summary and asks for confirmation

### Persistent Memory
- Stores patient records across sessions
- Tracks symptoms, diagnoses, and recommendations
- Maintains visit history with timestamps

### Professional Workflow
- Collects demographics first (Name, Age, Gender)
- Asks symptom-specific diagnostic questions
- Provides recommendations and doctor referrals when needed

### Key Benefits

#### Zero Code Maintenance
- Agent configured remotely in Flotorch Console
- Update prompts without touching code
- Memory provider managed centrally

#### Seamless Patient Experience
- Patients recognized automatically
- No manual data entry required
- Continuous care across visits

### Data Storage Format

Patient information is automatically stored as:
```json
{
  "Name": "John Doe",
  "Age": 35,
  "Gender": "Male",
  "Symptoms": "Headache, fatigue",
  "Diagnosis": "Tension headache",
  "Medical History": "No prior conditions",
  "Current Medications": "None",
  "Allergies": "None known",
  "Dietary Recommendations": "Stay hydrated, reduce caffeine",
  "Lifestyle Advice": "Regular sleep schedule, stress management",
  "Last Visit": "2025-10-30"
}
```

### Next Steps

1. **Customize**: Update agent behavior in Console → Agents → medi-assist
2. **Scale**: Add more patients and sessions as needed
3. **Monitor**: Check logs in Console → Logs tab

---

## 📚 Resources

- [Flotorch Documentation](https://docs.flotorch.cloud)
- [Agent Configuration Guide](https://console.flotorch.cloud/docs/agents)
- [Memory Service API](https://console.flotorch.cloud/docs/memory)